***Early Stopping***

In [2]:
import copy


class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        # number of epochs to wait for improvement before stopping the training.
        self.patience = patience
        # Minimum change in the monitored quantity validation loss to be considered as an improvement.
        self.min_delta = min_delta
        #A boolean flag indicating whether to restore the model's weights to the best observed during training when early stopping is triggered.
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        #1- Initialization Check:
        if self.best_loss is None:
            #it means that this is the first time the method is being called. In this case, it initializes self.best_loss with the current val_loss
            self.best_loss = val_loss
            #it deep copies the state of the model (model.state_dict()) into self.best_model.
            self.best_model = copy.deepcopy(model.state_dict())
            
        #2- Improvement Check:
        elif self.best_loss - val_loss >= self.min_delta:
            #Indicates an improvement in validation loss greater than or equal to the minimum delta required for considering it as an improvement. 
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            #Resets the counter to 0, and updates the status message.
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        #3- No Improvement:

        else:
            #ncrements the self.counter to indicate another epoch without improvement.
            self.counter += 1
            
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                #If the counter reaches or exceeds the patience threshold (self.patience), it triggers early stopping.
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    #if self.restore_best_weights is True, it loads the best model weights (self.best_model) back to the model.
                    model.load_state_dict(self.best_model)
                return True #Indicating that early stopping has been triggered.
        return False

***Drop_Out***

In [3]:
import numpy as np

class Dropout():
    def __init__(self, p):
        self.mask = None
        self.p = p
    
    def __call__(self, X, mode):
        return self.forward(X, mode)
    
    def forward(self, X, mode):
        if mode == 'train':
            self.mask = np.random.binomial(1, self.p, x.shape)
        #np.random.binomial is a NumPy function that generates random numbers from a binomial distribution.
        #The first argument 1 specifies that each element in the output will be either 0 or 1 (success or failure).
        #The second argument self.p represents the probability of success for each trial (the probability of keeping a neuron active during dropout).
        #The third argument x.shape determines the shape of the output array, which is the same shape as the input X.
        #This line generates a binary mask where each element has a value of 1 with probability self.p and 0 with probability (1 - self.p).
        
            self.mask = np.true_divide(self.mask, self.p)
        #p.true_divide is a NumPy function that performs element-wise division.
        #This line scales the mask by dividing each element by self.p.
        #Scaling by self.p ensures that the expected value of the output remains the same, effectively compensating for the dropout effect during the forward pass.
            out =  self.mask * X
        else:
            out = X
     
        return out
    
    def backward(self, d_out):
        return d_out * self.mask
        #This ensures that only the active neurons (those that weren't zeroed out during the forward pass) contribute to the gradients.

***L1-Regularization (LASSO)***

In [ ]:
# creating a class for Lasso Regression(L1-Regularization)

class Lasso_Regression():

  #initiating the hyperparameters
  def __init__(self, learning_rate, no_of_iterations, lambda_parameter):

    self.learning_rate = learning_rate
    self.no_of_iterations = no_of_iterations
    self.lambda_parameter = lambda_parameter


  # fitting the dataset to the Lasso Regression model
  def fit(self, X, Y):

    # m --> number of Data points --> number of rows
    # n --> number of input features --> number of columns
    self.m, self.n = X.shape

    self.w = np.zeros(self.n)

    self.b = 0

    self.X = X

    self.Y = Y

    # implementing Gradient Descent algorithm for Optimization

    for i in range(no_of_iterations):
      self.upadte_weights()


  # function for updating the weight & bias value
  def upadte_weights(self):

    # linear equation of the model
    Y_prediction = self.predict(self.X)

    # gradients (dw, db)

    # gradient for weight
    dw = np.zeros(self.n)

    for i in range(self.n):

      if self.w[i]>0:

        dw[i] = (-(2*(self.X[:,i]).dot(self.Y - Y_prediction)) + self.lambda_parameter) / self.m 

      else :

        dw[i] = (-(2*(self.X[:,i]).dot(self.Y - Y_prediction)) - self.lambda_parameter) / self.m


    # gradient for bias
    db = - 2 * np.sum(self.Y - Y_prediction) / self.m


    # updating the weights & bias

    self.w = self.w - self.learning_rate*dw
    self.b = self.b - self.learning_rate*db

    


  # Predicting the Target variable
  def predict(self,X):

    return X.dot(self.w) + self.b


